In [ ]:
from pynq import PL
from pynq import Overlay
from pynq import MMIO
import numpy as np
import matplotlib.pyplot as plt
import wave
from IPython.display import Audio as IPAudio
import audioop
from soundfile import SoundFile
from aip import AipSpeech
from getpass import getpass
from respeaker import *

In [ ]:
ol = Overlay("./bitstream/respeaker_wifi.bit")
ol.download()
ac108_init()
RESPEAKER_ADDR = 0x43C00000
RESPEAKER_RANGE = 0x1000
RESPEAKER_OFFSET = 0x00

APP_ID = getpass('APP ID: ')
API_KEY = getpass('API KEY: ')
SECRET_KEY = getpass('SECRET KEY: ')
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)

In [ ]:
mmio = MMIO(RESPEAKER_ADDR, RESPEAKER_RANGE)
cap_cnt = 44100
ch1 = np.zeros(shape=(cap_cnt),dtype=np.uint32)

In [ ]:
Wave_write1 = wave.open(r"recong.wav", 'w')
Wave_write1.setnchannels(1)
Wave_write1.setsampwidth(4)
Wave_write1.setframerate(cap_cnt)
for t in range(0,5):
    for i in range(0,cap_cnt):
        ch1[i] = mmio.read(0)
    Wave_write1.writeframes(ch1.tobytes())
Wave_write1.close()

In [ ]:
IPAudio("say_something.wav")

In [ ]:
file = SoundFile('say_something.wav')
temp_data = bytes(cap_cnt * 4)
file.buffer_read_into(temp_data, dtype='int16');
data = audioop.ratecv(temp_data, 2, 1, 44100, 16000, None)

In [ ]:
result = client.asr(data[0], 'pcm', 16000, {
    'dev_pid': 1737,
})
print(result)